In [1]:
import os
import six
import glob
import pickle

import numpy as np
import pandas as pd
import scipy.stats as spst

import p2pspatial
import pulse2percept.utils as p2pu

from sklearn.base import clone
import sklearn.metrics as sklm

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

/home/mbeyeler/anaconda3/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)
2018-04-07 21:54:52,482 [pulse2percept] [INFO] Welcome to pulse2percept


# Load data

In [2]:
def load_file(pickle_file, verbose=False):
    if verbose:
        print('- Processing %s' % pickle_file)
    _, _, _, specifics = pickle.load(open(pickle_file, 'rb'))
    if isinstance(specifics, list):
        print('List of specifics found in', pickle_file)
        return None
    row = {
        'subject': specifics['subject'],
        'model': specifics['modelname'],
        'exetime': specifics['exetime'],
        'adjust_bias': specifics['adjust_bias'],
        'n_folds': specifics['n_folds'],
        'idx_fold': specifics['idx_fold'],
        'best_cost': specifics['best_train_score'],
        'filepath': os.path.dirname(pickle_file),
        'filename': os.path.basename(pickle_file)
    }
    return row

In [3]:
results_dir = '../../results/shape3cv/'
pickle_files = np.sort(glob.glob(os.path.join(results_dir, '*.pickle')))
print('Found', len(pickle_files), 'files')

Found 1141 files


In [4]:
subjects = ['12-005', '51-009', '52-001', 'TB']
assert_params = {
    'amplitude': 2.0,
}
rootfolder = os.path.join(os.environ['SECOND_SIGHT_DATA'], 'shape')

In [5]:
data = pd.DataFrame(list(filter(None, p2pu.parfor(load_file, pickle_files))))

# All runs

In [6]:
# Group by these:
groupcols = ['subject', 'model', 'adjust_bias', 'idx_fold']
# Don't show these:
extracols = ['filepath', 'exetime', 'filename']
r2cols = ['r2_area', 'r2_orientation', 'r2_eccentricity', 'r2_compactness']

In [7]:
data.groupby(groupcols)['exetime'].agg(['mean', 'count'])

mean  count
subject model adjust_bias idx_fold                     
12-005  A     True        0           431.473700     10
                          1           364.866903     10
                          2           254.916891     10
                          3           224.855537     10
                          4           494.697876     10
                          5           359.697818     10
                          6           319.071494     10
                          7           362.117683     10
                          8           279.848912     10
        B     False       0           422.243781      5
                          1           366.484079      5
                          2           371.850290      5
                          3           516.704805      5
                          4           472.505403      5
                          5           482.252915      5
                          6           420.100409      5
                          7           327.683034      5
                          8           429.320474      5
              True        0           383.493953      5
                          1           298.848726      5
                          2           296.379649      5
                          3           362.378830      5
                          4           281.826416      5
                          5           347.633099      5
                          6           362.942974      5
                          7           356.517902      5
                          8           338.895019      5
        C     False       0         19163.691567      5
                          1         17098.481156      5
              True        0         13981.292982      5
...                                          ...    ...
TB      C     False       4          9821.984509      3
                          5          9981.868230      3
                          6         12861.295101      1
              True        0         20800.173811      5
                          1         16525.108954      5
                          2         10561.165242      3
                          3         12860.008166      3
                          4         20777.598683      2
                          5         11621.674028      2
                          6         14006.044015      3
                          7         14567.376285      3
                          8         14958.999099      3
        D     False       0         11904.340135      3
                          1         16185.673105      3
                          2         18029.169702      3
                          3         16871.230628      3
                          4         15770.646396      3
                          5         12118.031742      3
                          6         17371.765634      3
                          7         14664.673895      3
                          8         12707.705683      3
              True        0         15323.777730      3
                          1         14094.268002      3
                          2         20849.908298      3
                          3         20273.847335      3
                          4         10830.599441      3
                          5         25074.977316      3
                          6         19258.146722      3
                          7         14876.931044      3
                          8         15597.349977      3

[250 rows x 2 columns]

# The best runs in their respective category

In [8]:
print('Best scores:')
# Find the rows that have the best score in their subject/model/bias group:
best_idx = data.groupby(groupcols)['best_cost'].transform(np.min) == data['best_cost']
data.loc[best_idx, :].drop(columns=extracols).groupby(groupcols).max()

Best scores:


best_cost  n_folds
subject model adjust_bias idx_fold                    
12-005  A     True        0          8.066014       18
                          1          8.372955       18
                          2          8.273124       18
                          3          7.937856       18
                          4          8.013519       18
                          5          8.556084       18
                          6          7.783529       18
                          7          7.971503       18
                          8          8.771234       18
        B     False       0          4.587605       19
                          1          4.582119       19
                          2          5.124596       19
                          3          4.507815       19
                          4          4.387466       19
                          5          4.573606       19
                          6          4.396230       19
                          7          4.198560       19
                          8          4.889718       19
              True        0          4.513237       18
                          1          4.276810       18
                          2          4.544640       18
                          3          4.284552       18
                          4          4.321962       18
                          5          4.371777       18
                          6          4.311328       18
                          7          4.234962       18
                          8          4.451335       18
        C     False       0          4.491378       19
                          1          5.001893       19
              True        0          4.802035       18
...                                       ...      ...
TB      C     False       4          4.021852       11
                          5          4.225727       11
                          6          4.444174       11
              True        0          3.622852        9
                          1          3.282625        9
                          2          4.450765       10
                          3          3.090734       10
                          4          3.513777       10
                          5          3.551033       10
                          6          3.441179       10
                          7          3.590100       10
                          8          3.496361       10
        D     False       0          3.946496       10
                          1          4.294697       10
                          2          5.542475       10
                          3          4.556355       10
                          4          4.389111       10
                          5          4.557969       10
                          6          4.618428       10
                          7          5.446035       10
                          8          5.273489       10
              True        0          3.800470        9
                          1          3.886651        9
                          2          5.207910        9
                          3          4.582736        9
                          4          4.101841        9
                          5          4.670137        9
                          6          4.666913        9
                          7          4.730472        9
                          8          6.852698        9

[250 rows x 2 columns]

# Runs that give sufficiently good results

Predicts all four parameters:

In [9]:
best = data.loc[best_idx, :]
best4_idx = (best['r2_area'] > 0) & (best['r2_orientation'] > 0) & (best['r2_eccentricity'] > 0) & (best['r2_compactness'] > 0)
best.loc[best4_idx, :].drop(columns=extracols[:-1]).groupby(groupcols).max()

KeyError: 'r2_area'

Predicts at least 3 out of 4:

In [ ]:
best3_idx = (best['r2_area'] > 0) & (best['r2_orientation'] > 0) & (best['r2_eccentricity'] > 0)
best3_idx |= (best['r2_area'] > 0) & (best['r2_orientation'] > 0) & (best['r2_compactness'] > 0)
best3_idx |= (best['r2_eccentricity'] > 0) & (best['r2_orientation'] > 0) & (best['r2_compactness'] > 0)
best3_idx |= (best['r2_area'] > 0) & (best['r2_eccentricity'] > 0) & (best['r2_compactness'] > 0)
best.loc[best3_idx, :].drop(columns=extracols[:-1]).groupby(groupcols).max()

Predicts at least 2 out of 4:

In [ ]:
best2_idx = (best['r2_area'] > 0) & (best['r2_orientation'] > 0)
best2_idx |= (best['r2_area'] > 0) & (best['r2_compactness'] > 0)
best2_idx |= (best['r2_area'] > 0) & (best['r2_eccentricity'] > 0)
best2_idx |= (best['r2_orientation'] > 0) & (best['r2_compactness'] > 0)
best2_idx |= (best['r2_orientation'] > 0) & (best['r2_eccentricity'] > 0)
best2_idx |= (best['r2_eccentricity'] > 0) & (best['r2_compactness'] > 0)
best.loc[best2_idx, :].drop(columns=extracols[:-1]).groupby(groupcols).max()
best.loc[best2_idx, :].drop(columns=extracols[:-1]).groupby(groupcols).max()

In [ ]:
plot_files = best.loc[best2_idx, :].reset_index()

In [ ]:
y, _, _, _ = pickle.load(open(os.path.join(*plot_files.loc[plot_files.index[0], ['filepath', 'filename']]), 'rb'))
if isinstance(y, list):
    y = pd.concat(y)
columns = y.drop(columns=['electrode', 'image']).columns
columns

# Plot best ones

In [ ]:
fig, axes = plt.subplots(ncols=len(columns), nrows=len(plot_files), figsize=(14, 2 * len(plot_files)))
for (_, row), axrow in zip(plot_files.iterrows(), axes):
    y, y_pred, _, _ = pickle.load(open(os.path.join(*row[['filepath', 'filename']]), 'rb'))
    if isinstance(y, list):
        y = pd.concat(y)
        y_pred = pd.concat(y_pred)
    for col, ax in zip(columns, axrow):
        ax.scatter(y[col], y_pred[col])
        minval = np.minimum(y[col].min(), y_pred[col].min())
        maxval = np.maximum(y[col].max(), y_pred[col].max())
        ax.plot([minval, maxval], [minval, maxval], 'k--')
        ax.set_xlabel(col)
        ax.text(minval, maxval, "$R^2$=%.3f" % float(row['r2_' + col]), va='top')
    axrow[0].set_ylabel('%s %s %s' % (row['subject'], row['model'],
                                      "adjust" if row['adjust_bias'] else ""))
fig.tight_layout()

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [ ]:
y, y_pred, _, _ = pickle.load(open(os.path.join(*row[['filepath', 'filename']]), 'rb'))

In [ ]:
for col in columns:
    df = pd.DataFrame({'y': y[col], 'y_hat': y_pred[col]})
    model = ols('y ~ y_hat', data=df).fit()
    anovaResults = anova_lm(model)
    print('')
    print(col)
    print(anovaResults)